In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

from torchnlp.datasets import penn_treebank_dataset
import torch
from torchnlp.samplers import BPTTBatchSampler
from torch.utils.data import DataLoader
from rsm_samplers import MNISTSequenceSampler, ptb_pred_sequence_collate
from ptb_lstm import LSTMModel
import lang_util
from torchvision import datasets, transforms
import torch.nn.functional as F
import torchvision.utils as vutils
from torch.nn import CrossEntropyLoss
from importlib import reload 
from torch.utils.data import Sampler, BatchSampler
import rsm
from functools import reduce, partial
import matplotlib.pyplot as plt

In [34]:
reload(lang_util)
corpus = lang_util.Corpus('/Users/jgordon/nta/datasets/PTB')

print(corpus.read_out(corpus.train[:60]))

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 

 pierre <unk> N years old will join the board as a nonexecutive director nov. N 

 mr. <unk> is chairman of <unk> n.v. the dutch publishing group 

init tensor([0, 0, 0,  ..., 0, 0, 0]) tensor(0)
ids tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24])
ids tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 27, 24])
ids tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 27, 24, 39, 26, 40, 41, 42, 26, 43, 32, 44, 45, 46, 24])
ids tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14,

In [ ]:
torch.LongTensor(5)

In [ ]:
corpus.dictionary.idx2word[24]

In [108]:
import rsm_samplers
import rsm
import lang_util
reload(rsm_samplers)
reload(rsm)
reload(lang_util)

BS = 2
VS = 10000
EMB_DIM = 28
m=200
n=6

MB = 3

bwe = lang_util.BitwiseWordEmbedding()

model = rsm.RSMLayer(m=m, n=n, d_in=EMB_DIM, d_out=EMB_DIM, embed_dim=EMB_DIM)
collate_fn = partial(rsm_samplers.ptb_pred_sequence_collate, vector_dict=bwe.embedding_dict)

sampler = rsm_samplers.PTBSequenceSampler(corpus.train, batch_size=BS, max_batches=MB)
loader = DataLoader(corpus.train,
                       batch_sampler=sampler,
                       collate_fn=collate_fn)


In [114]:
x_b = torch.zeros((BS, m*n), dtype=torch.float32, requires_grad=False)
phi = torch.zeros((BS, m*n), dtype=torch.float32, requires_grad=False)
psi = torch.zeros((BS, m*n), dtype=torch.float32, requires_grad=False)
hidden = (x_b, phi, psi)

EPOCHS = 2

for i in range(EPOCHS):
    for batch_id, (inputs, target, pred_target, pred_input) in enumerate(loader):
        word = pred_input[0].item()
        print(word, corpus.read_out(word))
        x_a, hidden = model(inputs, hidden)
    print('...')

383 can
108 in
32 the
...
8207 kitchen
6762 walked
35 a
...


In [57]:
print(corpus.train[40:50])
print(corpus.read_out(corpus.train[38:50]))

tensor([24, 39, 26, 40, 41, 42, 26, 43, 32, 44])
nov. N <eos> mr. <unk> is chairman of <unk> n.v. the dutch


In [ ]:
import numpy as np

# start = np.random.randint(10000)
start = 0
for word_id in corpus.train[start:start+60]:
    print(word_id)
    print(corpus.read_out(word_id.item()))

In [ ]:
class BitwiseWordEmbedding(object):

    def __init__(self, vocab_size=10000, dim=28):
        self.vocab_size = vocab_size
        self.embedding_dict = {}
        self.dim = dim

    def generate_embeddings(self):
        for i in range(self.vocab_size):
            self.embedding_dict[i] = self.embed(i)

    def embed(self, i):
        first = "{0:b}".format(i).zfill(self.dim // 2)
        return first + self.inverse(first)

    def inverse(self, binstr):
        return ''.join('1' if x == '0' else '0' for x in binstr)

bwe = BitwiseWordEmbedding()

bwe.embed(9381)

In [121]:
from util import count_parameters

model = rsm.RSMLayer(m=600, n=8, d_in=EMB_DIM, d_out=EMB_DIM, embed_dim=EMB_DIM)

print(count_parameters(model))

model = rsm.RSMLayer(m=5000, n=1, d_in=EMB_DIM, d_out=EMB_DIM, embed_dim=EMB_DIM, fpartition=[0.25, 0, 0.75])

print(count_parameters(model))

23079028
18930028
